# Day 07: Amplifiers

## Grab input

In [32]:
from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.WARNING, datefmt='%I:%M:%S')

In [33]:
def grab_input(input_filename):
    with open(input_filename) as f:
        lines = f.read()
    output = list(map(int, lines.split(',')))
    return output

In [34]:
puzzle_input = grab_input("input")
puzzle_input

[3,
 8,
 1001,
 8,
 10,
 8,
 105,
 1,
 0,
 0,
 21,
 46,
 59,
 84,
 93,
 102,
 183,
 264,
 345,
 426,
 99999,
 3,
 9,
 1002,
 9,
 4,
 9,
 1001,
 9,
 3,
 9,
 102,
 2,
 9,
 9,
 1001,
 9,
 5,
 9,
 102,
 3,
 9,
 9,
 4,
 9,
 99,
 3,
 9,
 1002,
 9,
 3,
 9,
 101,
 4,
 9,
 9,
 4,
 9,
 99,
 3,
 9,
 1002,
 9,
 4,
 9,
 1001,
 9,
 4,
 9,
 102,
 2,
 9,
 9,
 1001,
 9,
 2,
 9,
 1002,
 9,
 3,
 9,
 4,
 9,
 99,
 3,
 9,
 1001,
 9,
 5,
 9,
 4,
 9,
 99,
 3,
 9,
 1002,
 9,
 4,
 9,
 4,
 9,
 99,
 3,
 9,
 101,
 2,
 9,
 9,
 4,
 9,
 3,
 9,
 102,
 2,
 9,
 9,
 4,
 9,
 3,
 9,
 1001,
 9,
 1,
 9,
 4,
 9,
 3,
 9,
 1001,
 9,
 1,
 9,
 4,
 9,
 3,
 9,
 1002,
 9,
 2,
 9,
 4,
 9,
 3,
 9,
 101,
 1,
 9,
 9,
 4,
 9,
 3,
 9,
 1001,
 9,
 2,
 9,
 4,
 9,
 3,
 9,
 102,
 2,
 9,
 9,
 4,
 9,
 3,
 9,
 1001,
 9,
 2,
 9,
 4,
 9,
 3,
 9,
 1002,
 9,
 2,
 9,
 4,
 9,
 99,
 3,
 9,
 101,
 1,
 9,
 9,
 4,
 9,
 3,
 9,
 102,
 2,
 9,
 9,
 4,
 9,
 3,
 9,
 1001,
 9,
 2,
 9,
 4,
 9,
 3,
 9,
 101,
 2,
 9,
 9,
 4,
 9,
 3,
 9,
 101,
 2,
 9,
 9,
 4,
 9,
 3

In [35]:
from itertools import permutations
combination_keys = [0, 1, 2, 3, 4]
possible_combinations = list(permutations(combination_keys, 5))

## Part 1

Based on the navigational maps, you're going to need to send more power to your ship's thrusters to reach Santa in time. To do this, you'll need to configure a series of amplifiers already installed on the ship.

There are five amplifiers connected in series; each one receives an input signal and produces an output signal. They are connected such that the first amplifier's output leads to the second amplifier's input, the second amplifier's output leads to the third amplifier's input, and so on. The first amplifier's input value is 0, and the last amplifier's output leads to your ship's thrusters.

    O-------O  O-------O  O-------O  O-------O  O-------O
0 ->| Amp A |->| Amp B |->| Amp C |->| Amp D |->| Amp E |-> (to thrusters)
    O-------O  O-------O  O-------O  O-------O  O-------O

The Elves have sent you some Amplifier Controller Software (your puzzle input), a program that should run on your existing Intcode computer. Each amplifier will need to run a copy of the program.

When a copy of the program starts running on an amplifier, it will first use an input instruction to ask the amplifier for its current phase setting (an integer from 0 to 4). Each phase setting is used exactly once, but the Elves can't remember which amplifier needs which phase setting.

The program will then call another input instruction to get the amplifier's input signal, compute the correct output signal, and supply it back to the amplifier with an output instruction. (If the amplifier has not yet received an input signal, it waits until one arrives.)

Your job is to find the largest output signal that can be sent to the thrusters by trying every possible combination of phase settings on the amplifiers. Make sure that memory is not shared or reused between copies of the program.

For example, suppose you want to try the phase setting sequence 3,1,2,4,0, which would mean setting amplifier A to phase setting 3, amplifier B to setting 1, C to 2, D to 4, and E to 0. Then, you could determine the output signal that gets sent from amplifier E to the thrusters with the following steps:

    Start the copy of the amplifier controller software that will run on amplifier A. At its first input instruction, provide it the amplifier's phase setting, 3. At its second input instruction, provide it the input signal, 0. After some calculations, it will use an output instruction to indicate the amplifier's output signal.
    Start the software for amplifier B. Provide it the phase setting (1) and then whatever output signal was produced from amplifier A. It will then produce a new output signal destined for amplifier C.
    Start the software for amplifier C, provide the phase setting (2) and the value from amplifier B, then collect its output signal.
    Run amplifier D's software, provide the phase setting (4) and input value, and collect its output signal.
    Run amplifier E's software, provide the phase setting (0) and input value, and collect its output signal.

The final output signal from amplifier E would be sent to the thrusters. However, this phase setting sequence may not have been the best one; another sequence might have sent a higher signal to the thrusters.

Here are some example programs:

    Max thruster signal 43210 (from phase setting sequence 4,3,2,1,0):

    3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0

    Max thruster signal 54321 (from phase setting sequence 0,1,2,3,4):

    3,23,3,24,1002,24,10,24,1002,23,-1,23,
    101,5,23,23,1,24,23,23,4,23,99,0,0

    Max thruster signal 65210 (from phase setting sequence 1,0,4,3,2):

    3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,
    1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0

Try every combination of phase settings on the amplifiers. What is the highest signal that can be sent to the thrusters?

In [36]:
# Define operations
def pad_modes(modes, size):
    while len(modes) < size:
        modes = '0' + modes
    logging.info(f"Modes: {list(modes)}")
    return list(modes)

def get_params(modes, position, program):
    params = [0,0,0]
    modes = pad_modes(modes, len(params))[::-1]
        
    position += 1
    for i in range(len(params)):
        if position < len(program):
            if modes[i] == "0":
                params[i] = program[position]
            elif modes[i] == "1":
                params[i] = position
            position += 1
    return params, position
        

def op_add(modes, position, program):
    params, position = get_params(modes, position, program)
    logging.info(f"{program[params[0]] + program[params[1]]} saved to [{params[2]}]\t({program[params[0]]} + {program[params[1]]})")
    program[params[2]] = program[params[0]] + program[params[1]]
    return position, program

def op_multiply(modes, position, program):
    params, position = get_params(modes, position, program)
    logging.info(f"{program[params[0]] * program[params[1]]} saved to [{params[2]}]\t({program[params[0]]} * {program[params[1]]})")
    program[params[2]] = program[params[0]] * program[params[1]]
    return position, program

def jump_if_true(modes, position, program):
    params, position = get_params(modes, position, program)
    if program[params[0]] != 0:
        position = program[params[1]]
        logging.debug(f"PC jumped to [{program[params[1]]}]")
    else:
        position -= 1
    return position, program

def jump_if_false(modes, position, program):
    params, position = get_params(modes, position, program)
    if program[params[0]] == 0:
        position = program[params[1]]
        logging.debug(f"PC jumped to [{program[params[1]]}]")
    else:
        position -= 1
    return position, program

def less_than(modes, position, program):
    params, position = get_params(modes, position, program)
    result = 0
    if program[params[0]] < program[params[1]]:
        result = 1
    program[params[2]] = result
    logging.info(f"Less than check: {result} written to [{program[params[2]]}]")
    return position, program

def equals(modes, position, program):
    params, position = get_params(modes, position, program)
    result = 0
    if program[params[0]] == program[params[1]]:
        result = 1
    program[params[2]] = result
    logging.info(f"Equals check: {result} written to [{program[params[2]]}]")
    return position, program

def op_save(input_value, modes, position, program):
    params, position = get_params(modes, position, program)
    program[params[0]] = input_value
    logging.info(f"Value {input_value} saved to [{params[0]}]")
    position -= 2
    return position, program

def op_load(modes, position, program):
    params, position = get_params(modes, position, program)
    output = program[params[0]]
    logging.info(f"Value {output} loaded from [{program[params[0]]}]")
    position -= 2
    return output, position, program

In [37]:
op_codes = {
    "1":op_add,
    "2":op_multiply,
    "3":op_save,
    "4":op_load,
    "5":jump_if_true,
    "6":jump_if_false,
    "7":less_than,
    "8":equals
}

In [41]:
def run_program(program, inputs):
    
    logging.debug(f"Inputs = {inputs}")
    pc = 0
    op_code = int(str(program[pc])[-2:])
    output = 0

    while op_code != 99:

        logging.info(f"PC: {pc}\tOpcode: {op_code}\tRaw: {str(program[pc])}")
        modes = str(program[pc])[:-2]

        if op_code == 3:
            input_value = inputs.pop()
            pc, program = op_codes[str(op_code)](input_value, modes, pc, program)

        elif op_code == 4:
            output, pc, program = op_codes[str(op_code)](modes, pc, program)

        else:
            pc, program = op_codes[str(op_code)](modes, pc, program)

        op_code = int(str(program[pc])[-2:])

    return output

In [63]:
puzzle_input = grab_input("test12")

In [64]:
highest_output = 0
highest_output_combo = None

for combination in possible_combinations:
    amplifier_inputs = [0]
    output = 0
    for amplifier_stage in range(len(combination)):
        output = 0
        amplifier_inputs.insert(0, combination[amplifier_stage])
        
        program = list(puzzle_input)
        output = run_program(program, amplifier_inputs)
        amplifier_inputs.append(output)
        
    if output > highest_output:
        highest_output = output
        highest_output_combo = combination 
        
print(f"Output: {highest_output} - Combo: {highest_output_combo}")

Output: 97 - Combo: (0, 1, 2, 3, 4)


## Part 2